In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from models import Model_1, Model_2, Model_3
import utils

In [2]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [3]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? False


In [8]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

## Model 1

In [5]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_1().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 10, 10, 10]             900
             ReLU-12           [-1, 10, 10, 10]               0
      BatchNorm2d-13           [-1, 10, 10, 10]              20
          Dropout-14           [-1, 10,

In [15]:
model = Model_1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.31238338351249695 Batch_id=468 Accuracy=80.67: 100%|██████████████████████████| 469/469 [01:19<00:00,  5.92it/s]



Test set: Average loss: 0.2524, Accuracy: 9476/10000 (94.76%)

EPOCH: 2


Loss=0.2876855432987213 Batch_id=468 Accuracy=86.83: 100%|███████████████████████████| 469/469 [01:08<00:00,  6.84it/s]



Test set: Average loss: 0.1652, Accuracy: 9630/10000 (96.30%)

EPOCH: 3


...


Test set: Average loss: 0.0646, Accuracy: 9848/10000 (98.48%)

EPOCH: 14


Loss=0.3081066310405731 Batch_id=468 Accuracy=88.71: 100%|███████████████████████████| 469/469 [01:42<00:00,  4.56it/s]



Test set: Average loss: 0.0684, Accuracy: 9839/10000 (98.39%)

EPOCH: 15


Loss=0.3390810191631317 Batch_id=468 Accuracy=88.88: 100%|███████████████████████████| 469/469 [01:59<00:00,  3.92it/s]



Test set: Average loss: 0.0621, Accuracy: 9853/10000 (98.53%)



## Model 2

In [10]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 10, 24, 24]             160
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 10, 10, 10]             900
             ReLU-10           [-1, 10, 10, 10]               0
      BatchNorm2d-11           [-1, 10, 10, 10]              20
           Conv2d-12             [-1, 16, 8, 8]           1,440
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [-1, 1

In [12]:
model = Model_2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.04411003366112709 Batch_id=468 Accuracy=92.21: 100%|██████████████████████████| 469/469 [00:47<00:00,  9.82it/s]



Test set: Average loss: 0.0840, Accuracy: 9743/10000 (97.43%)

EPOCH: 2


Loss=0.028225289657711983 Batch_id=468 Accuracy=97.96: 100%|█████████████████████████| 469/469 [00:51<00:00,  9.18it/s]



Test set: Average loss: 0.0560, Accuracy: 9826/10000 (98.26%)

EPOCH: 3


...


Test set: Average loss: 0.0346, Accuracy: 9887/10000 (98.87%)

EPOCH: 14


Loss=0.009823241271078587 Batch_id=468 Accuracy=99.47: 100%|█████████████████████████| 469/469 [00:58<00:00,  8.03it/s]



Test set: Average loss: 0.0295, Accuracy: 9904/10000 (99.04%)

EPOCH: 15


Loss=0.003706095740199089 Batch_id=468 Accuracy=99.50: 100%|█████████████████████████| 469/469 [00:52<00:00,  8.92it/s]



Test set: Average loss: 0.0337, Accuracy: 9902/10000 (99.02%)



## Model 3

In [13]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_3().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 10, 24, 24]             160
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 10, 10, 10]             900
             ReLU-10           [-1, 10, 10, 10]               0
      BatchNorm2d-11           [-1, 10, 10, 10]              20
           Conv2d-12             [-1, 16, 8, 8]           1,440
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [-1, 1

In [14]:
model = Model_3().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.07542801648378372 Batch_id=468 Accuracy=92.16: 100%|██████████████████████████| 469/469 [01:03<00:00,  7.42it/s]



Test set: Average loss: 0.0783, Accuracy: 9765/10000 (97.65%)

EPOCH: 2


Loss=0.022002853453159332 Batch_id=468 Accuracy=97.96: 100%|█████████████████████████| 469/469 [00:54<00:00,  8.65it/s]



Test set: Average loss: 0.0452, Accuracy: 9867/10000 (98.67%)

EPOCH: 3


...


Test set: Average loss: 0.0246, Accuracy: 9924/10000 (99.24%)

EPOCH: 14


Loss=0.01219156663864851 Batch_id=468 Accuracy=99.48: 100%|██████████████████████████| 469/469 [01:09<00:00,  6.76it/s]



Test set: Average loss: 0.0251, Accuracy: 9928/10000 (99.28%)

EPOCH: 15


Loss=0.026589535176753998 Batch_id=468 Accuracy=99.50: 100%|█████████████████████████| 469/469 [01:00<00:00,  7.71it/s]



Test set: Average loss: 0.0264, Accuracy: 9918/10000 (99.18%)

